# 데이터 수집.

In [1]:
from selenium import webdriver
from pandas.io.html import read_html
import time
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

# 1.selenium을 이용한 데이터 수집

In [2]:
browser=webdriver.Chrome()
sleep_time=3
page_num=30
# 후스코드어 홈페이지 접속.
url="https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/8228/Stages/18685/PlayerStatistics/England-Premier-League-2020-2021"
browser.get(url)

In [3]:
#df.columns 생성 (후드 스코어의 수비,공격,패스 카테고리별로 기록)
#해보니 player가 두개 생성 되므로, 둘다 받아온 후 처리 할 예정.
df_Passing=pd.DataFrame(columns=['Player', 'Player.1', 'Apps', 'Mins','Assists','KeyP','AvgP','PS%','Crosses','LongB','ThrB','Rating'])
df_Defensive=pd.DataFrame(columns=['Player', 'Player.1', 'Apps', 'Mins','Tackles','Inter','Fouls','Offsides','Clear','Drb','Blocks','OwnG','Rating'])
df_Offensive = pd.DataFrame(columns=['Player', 'Player.1', 'Apps', 'Mins', 'Goals', 'Assists', 'SpG','KeyP', 'Drb', 'Fouled', 'Off','Disp','UnsTch','Rating'])

In [4]:
#Defensive 클릭
time.sleep(sleep_time)
Defensive=browser.find_element_by_xpath('//*[@id="stage-top-player-stats-options"]/li[2]/a')
Defensive.click()

In [5]:
#DataFrame에 데이터 입력.
for i in np.arange(page_num)+1:
    time.sleep(sleep_time)
    table= browser.find_element_by_xpath('//*[@id="statistics-table-defensive"]')
    table_html= table.get_attribute('innerHTML')
    df2=read_html(table_html)[0]
    df_Defensive=pd.concat([df_Defensive,df2],axis=0)
    browser.find_element_by_link_text('next').click()

In [6]:
#Offensive 클릭
time.sleep(sleep_time)
Defensive=browser.find_element_by_xpath('//*[@id="stage-top-player-stats-options"]/li[3]/a')
Defensive.click()

In [7]:
#DataFrame에 데이터 입력.
for i in np.arange(page_num)+1:
    time.sleep(sleep_time)
    table= browser.find_element_by_xpath('//*[@id="statistics-table-offensive"]')
    table_html= table.get_attribute('innerHTML')
    df3=read_html(table_html)[0]
    df_Offensive=pd.concat([df_Offensive,df3],axis=0)
    browser.find_element_by_link_text('next').click()

In [8]:
#Passing 클릭
time.sleep(sleep_time)
Passing=browser.find_element_by_xpath('//*[@id="stage-top-player-stats-options"]/li[4]/a')
Passing.click()

In [9]:
#DataFrame에 데이터 입력.
for i in np.arange(page_num)+1:
    time.sleep(sleep_time)
    table= browser.find_element_by_xpath('//*[@id="statistics-table-passing"]')
    table_html= table.get_attribute('innerHTML')
    df3=read_html(table_html)[0]
    df_Passing=pd.concat([df_Passing,df3],axis=0)
    browser.find_element_by_link_text('next').click()

# 2.데이터 전처리 과정

In [23]:
# 컬럼 수정 함수.
def process(data):
    
    # 인덱스 수정
    data= data.reset_index()
    data.drop(['index','Player'],axis=1,inplace=True)
    spl=data['Player.1'].str.split(',')
    
    #1.이름
    name=[]
    for i in range(len(spl)):
        a = spl[i][0]
        name.append(a)
    data['name']=name
    
    #2.나이
    age=[]
    for i in range(len(spl)):
        a = int(spl[i][1])
        age.append(a)
    data['age']=age
    
    #3.포지션이 2개인 선수와 1개인 선수가 존재하니 구분해서 나눈다
    position1=[]
    for i in range(len(spl)):
        a = spl[i][2]
        position1.append(a)
    data['position1']=position1
    position2=[]
    for i in range(len(spl)):
        if len(spl[i])>3:
            a = spl[i][2]
        else:
            a = np.nan
        position2.append(a)
    data['position2']=position2
    
    #4.원래의 정보  삭제.
    data.drop('Player.1',axis=1,inplace=True)
    
    return data

In [26]:
Defensive=process(df_Defensive)
Offensive=process(df_Offensive)
Passing=process(df_Passing)

In [48]:
# '-' 값을 0으로 처리하기

In [43]:
for i in np.arange(len(Defensive.columns)):
    Defensive.iloc[Defensive.iloc[:,i]=='-',i]=0

In [45]:
for i in np.arange(len(Offensive.columns)):
    Offensive.iloc[Offensive.iloc[:,i]=='-',i]=0

In [46]:
for i in np.arange(len(Passing.columns)):
    Passing.iloc[Passing.iloc[:,i]=='-',i]=0

In [58]:
# 컬럼 순서 재배치.

In [56]:
col1=Defensive.columns[-4:].to_list()
col2=Defensive.columns[:-4].to_list()
new_col=col1+col2
Defensive=Defensive[new_col]

In [59]:
col1=Offensive.columns[-4:].to_list()
col2=Offensive.columns[:-4].to_list()
new_col=col1+col2
Offensive=Offensive[new_col]

,name,age,position1,position2,Apps,Mins,Goals,Assists,SpG,KeyP,Drb,Fouled,Off,Disp,UnsTch,Rating
0,Harry Kane Tottenham,28,AM(C),AM(C),35,3087,23,14,3.9,1.4,1.5,1.7,0.4,1.5,1.8,7.79
1,Kevin De Bruyne Man City,30,M(CLR),M(CLR),23(2),2001,6,12,3.2,3.2,1.9,1.2,0,1.3,1.8,7.65
2,Jack Grealish Aston Villa,26,AM(CL),NaN,24(2),2184,6,10,1.9,3.1,2.5,4.2,0.3,1.5,2.1,7.56
3,Bruno Fernandes Man Utd,27,M(CLR),NaN,35(2),3110,18,12,3.3,2.6,0.6,1.4,0.1,1.0,1.5,7.43
4,Tomas Soucek West Ham,26,DMC,NaN,38,3420,10,1,1.7,0.5,0.4,1.1,0.1,0.6,1.1,7.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,Mark Noble West Ham,34,M(C),NaN,8(13),707,0,0,0,0.2,0.1,0.4,0,0.3,0.2,6.17
287,Rhian Brewster Sheff Utd,21,FW,NaN,12(15),1125,0,0,0.9,0.4,0.3,0.5,0.1,0.7,0.7,6.17
288,Karlan Grant West Brom,24,AM(CL),AM(CL),14(7),1133,1,0,0.8,0.1,0.4,0.6,0.3,0.8,1.5,6.15
289,Hal Robson-Kanu West Brom,32,AM(LR),AM(LR),2(17),549,2,0,0.7,0.3,0.3,0.5,0.2,0.8,1.6,6.13


In [60]:
col1=Passing.columns[-4:].to_list()
col2=Passing.columns[:-4].to_list()
new_col=col1+col2
Passing=Passing[new_col]

# 3.csv 파일로 저장하기.

In [64]:
Defensive.to_csv('EPL_2020-2021_Player_Defensive_data.csv',sep=',')
Offensive.to_csv('EPL_2020-2021_Player_Offensive_data.csv',sep=',')
Passing.to_csv('EPL_2020-2021_Player_Passing_data.csv',sep=',')

In [66]:
Passing

,name,age,position1,position2,Apps,Mins,Assists,KeyP,AvgP,PS%,Crosses,LongB,ThrB,Rating
0,Harry Kane Tottenham,28,AM(C),AM(C),35,3087,14,1.4,25.9,69.9,0.2,1.9,0.1,7.79
1,Kevin De Bruyne Man City,30,M(CLR),M(CLR),23(2),2001,12,3.2,50.6,81.7,1.6,2.6,0.2,7.65
2,Jack Grealish Aston Villa,26,AM(CL),NaN,24(2),2184,10,3.1,38.3,83.3,0.7,1.2,0.1,7.56
3,Bruno Fernandes Man Utd,27,M(CLR),NaN,35(2),3110,12,2.6,57.1,78.3,1.1,3.8,0.2,7.43
4,Tomas Soucek West Ham,26,DMC,NaN,38,3420,1,0.5,42.3,75.5,0,2.2,0,7.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,Mark Noble West Ham,34,M(C),NaN,8(13),707,0,0.2,20.5,88.6,0.1,1.3,0,6.17
287,Rhian Brewster Sheff Utd,21,FW,NaN,12(15),1125,0,0.4,7.7,74.4,0.1,0.3,0.1,6.17
288,Karlan Grant West Brom,24,AM(CL),AM(CL),14(7),1133,0,0.1,10.3,75.1,0,0.1,0,6.15
289,Hal Robson-Kanu West Brom,32,AM(LR),AM(LR),2(17),549,0,0.3,4.2,70.9,0,0.1,0,6.13
